In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import os
import pandas as pd
import scipy.stats
import sys
sys.path.insert(1, '/glade/u/home/lettier/analysis/')
import master_utils as myf
xr.set_options(keep_attrs=True)

In [2]:
myvariables = ['SST']
mytime = pd.date_range(start="1979-01-01",end="2018-12-31", freq='M')
eddir = '/glade/scratch/wriggles/archive/'
mydir = '/glade/scratch/lettier/NUDGEconcat/'

In [3]:
def wrangle_nudge (nudge_name, myvariables):
    myfiles = [eddir+nudge_name+'/ocn/hist/'+f for f in os.listdir(eddir+nudge_name+'/ocn/hist/') if '.h.' in f and 'nday' not in f]
    ds_a = xr.open_mfdataset(myfiles).isel(z_t=0)[myvariables]
    
    myfiles = [eddir+nudge_name+'_21C/ocn/hist/'+f for f in os.listdir(eddir+nudge_name+'_21C/ocn/hist/') if '.h.' in f and 'nday' not in f]
    ds_b = xr.open_mfdataset(myfiles).isel(z_t=0)[myvariables]
    ds = xr.concat([ds_a,ds_b],dim='time')
    ds['time'] = mytime
    ds['names'] = nudge_name
    ds = ds.set_coords('names')
    
    return ds

In [4]:
def wrangle_lens (e, myvariables):
    ledir = '/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE/ocn/proc/tseries/monthly/'

    ens = str(e)
    if e<10:
        ens = '0'+str(e)
    tmp = []
    print(ens)
    for var in myvariables:
        myfiles = sorted([ledir+var+'/'+f for f in os.listdir(ledir+var) if ('B20TRC5CNBDRD' in f or 'BRCP85C5CNBDRD' in f) and '0'+ens+'.pop.h.'+var in f ])
        myfiles = [f for f in myfiles if '.192001-199912.nc' not in f and '208101-210012.nc' not in f]
        print(myfiles)
        ds = xr.open_mfdataset(myfiles)[[var]].sel(time=slice('1979-02','2019-01'))
        ds['time'] = mytime
        tmp.append(ds)
    ds = xr.merge(tmp)
    ds['names'] = 'LENS'+ens
    ds = ds.set_coords('names')

    return ds

In [5]:
def get_spatial_stuff (ds):
    name = str(ds.names.values)
    print(name)
 
    clim = ds.groupby('time.month').mean(dim='time')
    for var in ds:
        clim = clim.rename({var:var+'_clim'})
    
    slist = []
    for s, seas in enumerate(['JJA','DJF','SON','MMA']):
        mydata = ds.where(ds['time.season'] == seas).groupby('time.year').mean(dim='time')
        slope, intercept, r_value, p_value, std_err = myf.linregress(np.arange(len(mytime)/12),mydata.load(),dim='year')
    
        for var in ds:
            slope[var].attrs['units'] = clim[var+'_clim'].attrs['units']+'/yr'
            p_value[var] = 100.*p_value[var]
            p_value[var].attrs['units'] = '%'
            slope = slope.rename({var:var+'_trend'})
            p_value = p_value.rename({var:var+'_p_value'})

            
        sds = xr.merge([slope, p_value])
        sds['season'] = seas
        sds = sds.set_coords('season')
        slist.append(sds)
    sds = xr.concat(slist, dim='season')
        
        
    ds = xr.merge([clim, sds])

    ds.attrs['desc'] = 'processed by Lettie Roach, April 2021'
    ds.to_netcdf(mydir+'processed/spatial_mean_trend/ocn/'+name+'.ocn_climtrend.1979-2018.nc')
    
    return ds    
    

In [ ]:
%%time
ds = get_spatial_stuff (wrangle_nudge('nudge_era_low', ['SST']))


In [ ]:
%%time
ds = get_spatial_stuff (wrangle_lens(1, myvariables))


In [6]:
obds = xr.open_dataset('/glade/work/lettier/HADISST/remap/HadISST_sst_remapcesmogrid.nc').sel(time=slice('1979','2018'))
print(obds)


<xarray.Dataset>
Dimensions:    (time: 480, nlat: 384, nlon: 320, nv: 2)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-16T12:00:00 ... 2018-12-16T12:00:00
    TLONG      (nlat, nlon) float64 ...
    TLAT       (nlat, nlon) float64 ...
Dimensions without coordinates: nlat, nlon, nv
Data variables:
    time_bnds  (time, nv) float32 ...
    sst        (time, nlat, nlon) float32 ...
Attributes:
    CDI:                        Climate Data Interface version 1.9.9 (https:/...
    Conventions:                CF-1.0
    source:                     HadISST
    institution:                Met Office Hadley Centre
    Title:                      Monthly version of HadISST sea surface temper...
    description:                HadISST 1.1 monthly average sea surface tempe...
    reference:                  Rayner, N. A., Parker, D. E., Horton, E. B., ...
    history:                    Fri Jan 07 14:06:53 2022: cdo remapbil,/glade...
    supplementary_information:  Updates and supplemen

In [7]:
obds.time

<xarray.DataArray 'time' (time: 480)>
array(['1979-01-16T12:00:00.000000000', '1979-02-15T00:00:00.000000000',
       '1979-03-16T12:00:00.000000000', ..., '2018-10-16T12:00:00.000000000',
       '2018-11-16T12:00:00.000000000', '2018-12-16T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1979-01-16T12:00:00 ... 2018-12-16T12:00:00
Attributes:
    standard_name:  time
    long_name:      Time
    axis:           T

In [8]:
obds['time'] = mytime
obds = obds.rename({'sst':'SST'})
obds['names'] = 'HADISST'
obds = obds.set_coords('names')
print(obds)

<xarray.Dataset>
Dimensions:    (time: 480, nlat: 384, nlon: 320, nv: 2)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-31 1979-02-28 ... 2018-12-31
    TLONG      (nlat, nlon) float64 ...
    TLAT       (nlat, nlon) float64 ...
    names      <U7 'HADISST'
Dimensions without coordinates: nlat, nlon, nv
Data variables:
    time_bnds  (time, nv) float32 ...
    SST        (time, nlat, nlon) float32 ...
Attributes:
    CDI:                        Climate Data Interface version 1.9.9 (https:/...
    Conventions:                CF-1.0
    source:                     HadISST
    institution:                Met Office Hadley Centre
    Title:                      Monthly version of HadISST sea surface temper...
    description:                HadISST 1.1 monthly average sea surface tempe...
    reference:                  Rayner, N. A., Parker, D. E., Horton, E. B., ...
    history:                    Fri Jan 07 14:06:53 2022: cdo remapbil,/glade...
    supplementary_information: 

In [10]:
ds = get_spatial_stuff (obds)

HADISST


KeyError: 'units'